## PARIS SUBSIDIES: DATA CLEANING (SIRENE)

### LIBRAIRIES IMPORT

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

### DATA IMPORT

In [3]:
SIRENE = pd.read_csv('../00_DataFiles/01_Collected/ParisSubsidies_SIRENE.csv')
SIRENE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11161 entries, 0 to 11160
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   fields.libellecommuneetablissement         11160 non-null  object 
 1   fields.categorieentreprise                 6609 non-null   object 
 2   fields.etatadministratifunitelegale        11161 non-null  object 
 3   fields.geolocetablissement                 11021 non-null  object 
 4   fields.trancheeffectifsunitelegale         6850 non-null   object 
 5   fields.trancheeffectifsunitelegaletriable  6850 non-null   float64
 6   fields.siretsiegeunitelegale               11161 non-null  int64  
 7   fields.siret                               11161 non-null  int64  
 8   fields.etablissementsiege                  11161 non-null  object 
 9   fields.adresseetablissement                11155 non-null  object 
 10  fields.sectionuniteleg

#### Profile report before cleaning

In [4]:
profile = ProfileReport(SIRENE.reset_index(drop=True), title="SIRENE Profiling Report - Before Cleaning")
profile.to_file("ProfileReports/ParisSubsidies_SIRENE_bf.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 459.35it/s]


### FIELD BY FIELD APPROACH

### Duplicates

In [74]:
SIRENE.loc[SIRENE.duplicated(keep = False)].sort_values(by='fields.siret')

,fields.libellecommuneetablissement,fields.categorieentreprise,fields.geolocetablissement,fields.trancheeffectifsunitelegale,fields.trancheeffectifsunitelegaletriable,fields.siretsiegeunitelegale,fields.siret,fields.etablissementsiege,fields.adresseetablissement,fields.codepostaletablissement,fields.caractereemployeurunitelegale,fields.datecreationunitelegale,fields.datecreationetablissement,fields.denominationunitelegale
989,MONTREUIL,PME,"[48.86182, 2.435903]",20 à 49 salariés,12.0,31081902400293,31081902400293,oui,14 Rue DE LA BEAUNE,93100.0,Oui,1973-10-30,2017-12-01,VACANCES & FAMILLES
11159,MONTREUIL,PME,"[48.86182, 2.435903]",20 à 49 salariés,12.0,31081902400293,31081902400293,oui,14 Rue DE LA BEAUNE,93100.0,Oui,1973-10-30,2017-12-01,VACANCES & FAMILLES
11026,PARIS 11,PME,"[48.853229, 2.389652]",6 à 9 salariés,3.0,32079827500030,32079827500030,oui,10 Impasse DELEPINE,75011.0,Oui,1980-08-01,1993-07-09,BABILLAGES
6978,PARIS 11,PME,"[48.853229, 2.389652]",6 à 9 salariés,3.0,32079827500030,32079827500030,oui,10 Impasse DELEPINE,75011.0,Oui,1980-08-01,1993-07-09,BABILLAGES
11093,PARIS 6,PME,"[48.845433, 2.321548]",1 ou 2 salariés,1.0,32239771200017,32239771200017,oui,103 Rue DE VAUGIRARD,75006.0,Oui,1981-08-01,1981-08-01,SOS PARIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,PARIS 14,Non productif,"[48.829565, 2.329143]",Etablissement non employeur,-1.0,83479403400019,83479403400019,oui,47 Rue REMY DUMONCEL,75014.0,Non,2017-12-01,2017-12-01,"PARIS 14, TERRITOIRE DE CINEMA"
10994,PARIS 15,Non productif,"[48.835341, 2.30214]",Etablissement non employeur,-1.0,83526701400024,83526701400024,oui,60 Rue DOMBASLE,75015.0,Non,2010-04-28,2018-02-16,ASSOCIATION DES COMMERCANTS DE LA RUE VOUILLE
7846,PARIS 15,Non productif,"[48.835341, 2.30214]",Etablissement non employeur,-1.0,83526701400024,83526701400024,oui,60 Rue DOMBASLE,75015.0,Non,2010-04-28,2018-02-16,ASSOCIATION DES COMMERCANTS DE LA RUE VOUILLE
6962,PARIS 19,Non productif,"[48.873641, 2.377641]",Etablissement non employeur,-1.0,83834612000010,83834612000010,oui,15 Rue JULES ROMAINS,75019.0,Non,2017-03-29,2017-03-29,ESPOIR BERBERE


In [75]:
SIRENE.drop_duplicates(inplace = True)

### Missing values

In [5]:
SIRENE.isna().sum()

fields.libellecommuneetablissement              1
fields.categorieentreprise                   4552
fields.etatadministratifunitelegale             0
fields.geolocetablissement                    140
fields.trancheeffectifsunitelegale           4311
fields.trancheeffectifsunitelegaletriable    4311
fields.siretsiegeunitelegale                    0
fields.siret                                    0
fields.etablissementsiege                       0
fields.adresseetablissement                     6
fields.sectionunitelegale                       0
fields.codepostaletablissement                  1
fields.caractereemployeurunitelegale            0
fields.datecreationunitelegale                  0
fields.datecreationetablissement                0
fields.denominationunitelegale                  4
geometry.type                                 140
geometry.coordinates                          140
dtype: int64

### fields.libellecommuneetablissement 
We keep it for visualisation purposes (hover labels).

We might also need it to complete the NaNs of fields.geolocetablissement.

We just need to complete the missing data.

In [62]:
SIRENE.loc[SIRENE['fields.libellecommuneetablissement'].isna()]

,fields.libellecommuneetablissement,fields.categorieentreprise,fields.geolocetablissement,fields.trancheeffectifsunitelegale,fields.trancheeffectifsunitelegaletriable,fields.siretsiegeunitelegale,fields.siret,fields.etablissementsiege,fields.adresseetablissement,fields.sectionunitelegale,fields.codepostaletablissement,fields.caractereemployeurunitelegale,fields.datecreationunitelegale,fields.datecreationetablissement,fields.denominationunitelegale
7622,NaN,PME,"[49.028226, 3.957707]",10 à 19 salariés,11.0,31935615000015,31935615000015,oui,HOTEL DE VILLE QUEBEC,Autres activités de services,NaN,Oui,1980-01-01,1980-01-01,ASS INTERNATIONALE DES MAIRES FRANCOPHON


In [63]:
SIRENE.loc[7622, 'fields.libellecommuneetablissement'] = 'QUEBEC'
SIRENE.loc[7622, 'fields.codepostaletablissement'] = 'G1R 4S9'

### fields.categorieentreprise 
Many NaNs: they are 'non productive' entities (https://www.insee.fr/fr/information/1730869)

In [6]:
SIRENE['fields.categorieentreprise'].fillna('Non productif', inplace = True)

### fields.etatadministratifunitelegale
Almost all entities are still active so field is not very informative, we can drop it.

In [7]:
SIRENE.drop('fields.etatadministratifunitelegale', axis = 1, inplace = True)

### fields.geolocetablissement
We need to examine and potentially fill the missing values

In [8]:
SIRENE[SIRENE['fields.geolocetablissement'].isna()].head()

,fields.libellecommuneetablissement,fields.categorieentreprise,fields.geolocetablissement,fields.trancheeffectifsunitelegale,fields.trancheeffectifsunitelegaletriable,fields.siretsiegeunitelegale,fields.siret,fields.etablissementsiege,fields.adresseetablissement,fields.sectionunitelegale,fields.codepostaletablissement,fields.caractereemployeurunitelegale,fields.datecreationunitelegale,fields.datecreationetablissement,fields.denominationunitelegale,geometry.type,geometry.coordinates
216,PARIS 16,Non productif,NaN,NaN,NaN,83405071800016,83405071800016,oui,2 Avenue PAUL DOUMER,Autres activités de services,75116.0,Non,2017-10-11,2017-10-11,ASSOCIATION MANASSE,NaN,NaN
351,PARIS 16,PME,NaN,1 ou 2 salariés,1.0,81065441800013,81065441800013,oui,15 Avenue PRESIDENT WILSON,"Arts, spectacles et activités récréatives",75116.0,Oui,2007-05-15,2007-05-15,ASSOCIATION QI GONG EE TONG FRANCE,NaN,NaN
495,PARIS 16,PME,NaN,10 à 19 salariés,11.0,32765711000018,32765711000018,oui,71 Avenue HENRI MARTIN,"Arts, spectacles et activités récréatives",75116.0,Oui,1983-01-01,1983-01-01,CERCLE FEMININ PARIS,NaN,NaN
766,PARIS 16,PME,NaN,6 à 9 salariés,3.0,34088665400011,34088665400011,oui,34 Avenue DE NEW YORK,Autres activités de services,75116.0,Oui,1987-01-01,1987-01-01,AMERICAN CENTER FOR ART AND CULTURE,NaN,NaN
797,PARIS 16,PME,NaN,1 ou 2 salariés,1.0,78467027500022,78467027500022,oui,27 Rue DECAMPS,Autres activités de services,75116.0,Oui,1900-01-01,1900-01-01,AUMONERIE CATHOLIQUE JANSON-DELACROIX,NaN,NaN


In [9]:
# Are they all located in the XVIe arrondissement?
SIRENE.loc[SIRENE['fields.geolocetablissement'].isna(),'fields.codepostaletablissement'].value_counts().head()

75116.0    113
75004.0      4
91080.0      2
75015.0      2
76620.0      2
Name: fields.codepostaletablissement, dtype: int64

In [46]:
# Almost all of them: probably the geolocalisation was not able to deal with 75116 as 'code postal' instead of 75016.
# We will try to get the geolocation based on the address (switching 75116 to 75016).

# Subsetting
subset = SIRENE.loc[SIRENE['fields.geolocetablissement'].isna()].copy()
subset['fields.codepostaletablissement'] = np.where(subset['fields.codepostaletablissement'] == 75116, '75016', subset['fields.codepostaletablissement'].astype(int))

# Geocoding
geolocator = Nominatim(user_agent="ParisSubsidies")

for i in subset.index:
    d = {'street':subset.loc[i,'fields.adresseetablissement'], 'postalcode': subset.loc[i,'fields.codepostaletablissement'], 'country':'FRANCE'}
    p = geolocator.geocode(d)
    try:
        subset.loc[i,'fields.geolocetablissement'] = '[' + str(p.longitude) + ',' + str(p.latitude) + ']'
    except:
        pass

In [49]:
subset.loc[subset['fields.geolocetablissement'].isna(), ['fields.adresseetablissement', 'fields.codepostaletablissement', 'fields.denominationunitelegale']]

,fields.adresseetablissement,fields.codepostaletablissement,fields.denominationunitelegale
2768,DEPARTEMENTALE 128,91190,SYSTEM@TIC PARIS REGION
3297,8 Place LES COPAINS D ABORD,91080,LES PASSEURS D'ONDES
3480,1 Place MAL DE LATTRE DE TASSIGNY,75016,ASSOCIATION SPI DAUPHINE
4570,Place MAL DE LATTRE DE TASSIGNY,75016,OREILLE DE DAUPHINE
5055,1 Place MAL DE LATTRE DE TASSIGNY,75016,THEATRE A DAUPHINE
5156,55 Rue DU 329E RGT D'INFANTERIE,76620,ASSOCIATION LA BAZOOKA
5876,7 Place M RENAUD -JL BARRAULT,75015,LES DONNEURS DE VOIX
6587,PARC D'ACTIVITES,84120,ACIDD
6954,5 Avenue LA GRANDE ARMEE,75016,CENTRE DE RECHERCHE INTERDISCIPLINAIRE EN JURI...
7114,55 Rue DU 329E RGT D'INFANTERIE,76620,COMPAGNIE AKTE


In [57]:
# Manual input for remaining NaNs
subset.loc[2768, 'fields.geolocetablissement'] = '[48.6572354,2.18586]'
subset.loc[3297, 'fields.geolocetablissement'] = '[48.6287341,2.4156858]'
subset.loc[3480, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'
subset.loc[4570, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'
subset.loc[5055, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'
subset.loc[5156, 'fields.geolocetablissement'] = '[49.5039949,0.122207]'
subset.loc[5876, 'fields.geolocetablissement'] = '[48.8383529,2.2972237]'
subset.loc[6587, 'fields.geolocetablissement'] = '[43.7033551,5.5627938]'
subset.loc[6954, 'fields.geolocetablissement'] = '[48.8742464,2.2901752]'
subset.loc[7114, 'fields.geolocetablissement'] = '[49.5039949,0.122207]'
subset.loc[7714, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'
subset.loc[7768, 'fields.geolocetablissement'] = '[47.3322116,5.0458991]'
subset.loc[7968, 'fields.geolocetablissement'] = '[48.9255848,2.3621314]'
subset.loc[9092, 'fields.geolocetablissement'] = '[48.8421348,2.3316201]'
subset.loc[9625, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'
subset.loc[10360, 'fields.geolocetablissement'] = '[48.8710273,2.271914]'

In [58]:
# Load data into SIRENE
for i in subset.index:
    SIRENE.loc[i, 'fields.geolocetablissement'] = subset.loc[i,'fields.geolocetablissement']

### fields.trancheeffectifsunitelegale & fields.trancheeffectifsunitelegaletriable 
NaNs are 'Etablissement non employeur' with triable = -1

In [61]:
SIRENE['fields.trancheeffectifsunitelegale'].fillna('Etablissement non employeur', inplace = True)
SIRENE['fields.trancheeffectifsunitelegaletriable'].fillna(-1, inplace = True)

### fields.adresseetablissement
We keep it for visualisation purposes (hover labels).

We just need to fill the NaNs.

In [64]:
SIRENE.loc[SIRENE['fields.adresseetablissement'].isna()]

,fields.libellecommuneetablissement,fields.categorieentreprise,fields.geolocetablissement,fields.trancheeffectifsunitelegale,fields.trancheeffectifsunitelegaletriable,fields.siretsiegeunitelegale,fields.siret,fields.etablissementsiege,fields.adresseetablissement,fields.sectionunitelegale,fields.codepostaletablissement,fields.caractereemployeurunitelegale,fields.datecreationunitelegale,fields.datecreationetablissement,fields.denominationunitelegale
1267,LES PILLES,PME,"[44.379526, 5.188954]",Etablissement non employeur,-1.0,41763712100019,41763712100019,oui,NaN,Autres activités de services,26110.0,Non,1997-11-01,1997-11-01,ASSOCIATION AFRICULTURES
4846,COEUVRES-ET-VALSERY,PME,"[49.335993, 3.151804]",1 ou 2 salariés,1.0,38750560500023,38750560500023,oui,NaN,"Arts, spectacles et activités récréatives",2600.0,Non,1991-06-25,2005-05-31,LA LANTERNE MAGIQUE
6327,LOUHOSSOA,PME,"[43.317797, -1.355818]",10 à 19 salariés,11.0,41103434100049,41103434100049,oui,NaN,"Arts, spectacles et activités récréatives",64250.0,Oui,1996-02-26,2008-09-20,LE PETIT THEATRE DE PAIN
7612,FLEURY-DEVANT-DOUAUMONT,Non productif,"[49.193794, 5.43252]",Etablissement non employeur,-1.0,78339244200019,78339244200019,oui,NaN,"Arts, spectacles et activités récréatives",55100.0,Non,1900-01-01,1900-01-01,COMITE DU MEMORIAL DE VERDUN
10836,PRASLAY,PME,"[47.74088, 5.105861]",1 ou 2 salariés,1.0,47957806400012,47957806400012,oui,NaN,"Arts, spectacles et activités récréatives",52160.0,Oui,2004-10-18,2004-10-18,LES DECISIFS
11123,SOURCE-SEINE,Non productif,"[47.490598, 4.686979]",Etablissement non employeur,-1.0,50097582600016,50097582600016,oui,NaN,Autres activités de services,21690.0,Non,2007-11-15,2007-11-15,ASSOCIATION DES SOURCES DE LA SEINE


In [65]:
SIRENE.loc[1267, 'fields.adresseetablissement'] = 'Le Village'
SIRENE.loc[4846, 'fields.adresseetablissement'] = '28 Rue Gabrielle d\'Estrées'
SIRENE.loc[6327, 'fields.adresseetablissement'] = 'Le Bourg'
SIRENE.loc[7612, 'fields.adresseetablissement'] = '1 Av. Corps Européen'
SIRENE.loc[10836, 'fields.adresseetablissement'] = '2 Rue des Chassaignes'
SIRENE.loc[11123, 'fields.adresseetablissement'] = '22 Rue de Saint-Germain'

In [84]:
# all in upper case
SIRENE['fields.adresseetablissement'] = SIRENE['fields.adresseetablissement'].str.upper()

### fields.sectionunitelegale 
Not very informative, we have better information in the subsidies file, we can drop it.

In [66]:
SIRENE.drop('fields.sectionunitelegale', axis = 1, inplace = True)

### fields.denominationunitelegale
Fill the NaNs

In [67]:
SIRENE.loc[SIRENE['fields.denominationunitelegale'].isna()]

,fields.libellecommuneetablissement,fields.categorieentreprise,fields.geolocetablissement,fields.trancheeffectifsunitelegale,fields.trancheeffectifsunitelegaletriable,fields.siretsiegeunitelegale,fields.siret,fields.etablissementsiege,fields.adresseetablissement,fields.codepostaletablissement,fields.caractereemployeurunitelegale,fields.datecreationunitelegale,fields.datecreationetablissement,fields.denominationunitelegale
1523,PARIS 20,PME,"[48.863613, 2.405414]",Etablissement non employeur,-1.0,50980367200013,50980367200013,oui,4 VLA DES LYANES,75020.0,Non,2009-01-09,2009-01-09,NaN
2498,PARIS 16,Non productif,"[48.837568, 2.259573]",Etablissement non employeur,-1.0,51265462500015,51265462500015,oui,10 Rue GUDIN,75016.0,Non,2009-06-02,2009-06-02,NaN
5732,PARIS 18,PME,"[48.890392, 2.345489]",Etablissement non employeur,-1.0,82122548900011,82122548900011,oui,59 Rue RAMEY,75018.0,Non,2016-07-01,2016-07-01,NaN
5886,PARIS 14,PME,"[48.819309, 2.343683]",Etablissement non employeur,-1.0,52837795500026,52837795500018,non,1 Boulevard JOURDAN,75014.0,Non,2009-03-16,2009-03-16,NaN


In [68]:
SIRENE.loc[1523, 'fields.denominationunitelegale'] = 'ALESSANDRA LE DU'
SIRENE.loc[2498, 'fields.denominationunitelegale'] = 'NADIA BARBE'
SIRENE.loc[5732, 'fields.denominationunitelegale'] = 'VERONIQUE RIEFFEL'
SIRENE.loc[5886, 'fields.denominationunitelegale'] = 'AHMED LAHLOU'

### geometry.type and geometry.coordinates
All points and duplicated information with fields.geolocetablissement so we can drop them

In [60]:
SIRENE.drop(['geometry.type', 'geometry.coordinates'], axis = 1, inplace = True)

### Columns name
We can remove all the 'fields.'

In [77]:
SIRENE.columns = SIRENE.columns.str.replace('fields.', '', regex=False)

In [78]:
SIRENE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11108 entries, 0 to 11156
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   libellecommuneetablissement         11108 non-null  object 
 1   categorieentreprise                 11108 non-null  object 
 2   geolocetablissement                 11108 non-null  object 
 3   trancheeffectifsunitelegale         11108 non-null  object 
 4   trancheeffectifsunitelegaletriable  11108 non-null  float64
 5   siretsiegeunitelegale               11108 non-null  int64  
 6   siret                               11108 non-null  int64  
 7   etablissementsiege                  11108 non-null  object 
 8   adresseetablissement                11108 non-null  object 
 9   codepostaletablissement             11108 non-null  object 
 10  caractereemployeurunitelegale       11108 non-null  object 
 11  datecreationunitelegale             11108

In [85]:
SIRENE.head()

,libellecommuneetablissement,categorieentreprise,geolocetablissement,trancheeffectifsunitelegale,trancheeffectifsunitelegaletriable,siretsiegeunitelegale,siret,etablissementsiege,adresseetablissement,codepostaletablissement,caractereemployeurunitelegale,datecreationunitelegale,datecreationetablissement,denominationunitelegale
0,PARIS 12,PME,"[48.843084, 2.38305]",3 à 5 salariés,2.0,31939212200036,31939212200036,oui,78 RUE DU CHAROLAIS,75012.0,Oui,1980-06-04,1994-04-01,CENTRE THEATRE DE L'OPPRIME AUGUSTO BOAL
1,PARIS 20,PME,"[48.854959, 2.397993]",6 à 9 salariés,3.0,78480996400033,78480996400033,oui,77 RUE ALEXANDRE DUMAS,75020.0,Oui,1900-01-01,2000-02-26,EDUCAT POPUL CHARONNE REUNION
2,PARIS 18,Non productif,"[48.886052, 2.360784]",Etablissement non employeur,-1.0,48880307300011,48880307300011,oui,19 RUE PAJOL,75018.0,Non,2005-12-15,2005-12-15,CANOPY
3,PARIS 15,PME,"[48.831203, 2.296779]",1 ou 2 salariés,1.0,40206283000045,40206283000045,oui,47 RUE DE DANTZIG,75015.0,Oui,1995-01-28,2013-02-02,DROITS DEVANT
4,PARIS 17,PME,"[48.895408, 2.321683]",3 à 5 salariés,2.0,79258420300014,79258420300014,oui,64 RUE POUCHET,75017.0,Oui,2011-09-05,2011-09-05,ASSOCIAITON MAISON DES VOLONTAIRES


## SAVE

### Profile report after cleaning

In [79]:
profile = ProfileReport(SIRENE.reset_index(drop=True), title="SIRENE Profiling Report - After Cleaning")
profile.to_file("ProfileReports/ParisSubsidies_SIRENE_af.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 251.90it/s]


### CSV

In [80]:
SIRENE.to_csv('../00_DataFiles/02_Cleaned/ParisSubsidies_SIRENE.csv', index = False)